In [1]:
import pandas as pd
import pickle
from pyvi.ViTokenizer import tokenize
import json

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
def normalize_text(text):
    from bs4 import BeautifulSoup
    import re
    from pyvi.ViTokenizer import tokenize
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text
    text = strip_html_tags(text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # emove duplicate spaces
    text = re.sub(r'  +', ' ', text)
    # remove duplicate newline characters if more than 2
    text = re.sub(r'\n{2,}', '\n', text)
    # remove all \n if at the end of the text
    text = re.sub(r'\n+$', '', text)
    # remove all \n if at the beginning of the text
    text = re.sub(r'^\n+', '', text)

    tokenize_text = tokenize(text)
    return tokenize_text

In [5]:
df = pd.read_json('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', lines=True)

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
df.head(5)

,id,url,title,text,timestamp,revid
0,2577327,https://vi.wikipedia.org/wiki?curid=2577327,Amplicincia lathyi,Amplicincia lathyi\n\nAmplicincia lathyi là mộ...,2021-12-21 04:35:33+00:00,67703402
1,1912208,https://vi.wikipedia.org/wiki?curid=1912208,Kniphofia thomsonii,Kniphofia thomsonii\n\nKniphofia thomsonii là ...,2021-10-17 21:19:56+00:00,66396059
2,1882686,https://vi.wikipedia.org/wiki?curid=1882686,Catasetum ollare,Catasetum ollare\n\nCatasetum ollare là một lo...,2021-12-20 00:59:12+00:00,67645152
3,1318025,https://vi.wikipedia.org/wiki?curid=1318025,Ciclova Română,Ciclova Română\n\nCiclova Română là một xã thu...,2022-02-07 11:43:35+00:00,68116159
4,887188,https://vi.wikipedia.org/wiki?curid=887188,Euryglossa cupreochalybea,Euryglossa cupreochalybea\n\nEuryglossa cupreo...,2021-11-04 12:53:09+00:00,66679020


In [8]:
df['text'] = df['text'].parallel_apply(normalize_text)

In [36]:
exam = df[df['title']=='Bệnh viện Grall']
print('title:',exam.title.values[0])
exam.text.values[0]

title: Bệnh viện Grall


'Bệnh viện Grall\nBệnh viện Grall là một bệnh viện lớn ở Sài Gòn hoạt động từ năm 1925 đến 1978.\n Lịch sử. \nTiền thân của Bệnh viện Grall là Bệnh viện Quân sự (tiếng Pháp: "Hôpital militaire") của Quân đội Pháp, thành lập từ năm 1862 khi họ mới xâm chiếm Nam Kỳ. Cơ sở này vào cuối thập niên 1870 chuyển về số 14 rue Lagrandière, tức địa điểm hiện nay. Tại cơ sở này nhà bác học Albert Calmette cho thành lập Viện Pasteur (Pasteur-Institut) đầu tiên ở ngoài nước Pháp năm 1891.\nCấu trúc các tòa nhà trong khuôn viên đều là sườn sắt tiền chế đem ráp lại trên nền bằng đá. Mọi vật liệu mang từ Pháp sang.\nTừ năm 1905 trở đi cơ sở y tế này dưới sự điều hành của bác sĩ Charles Grall, mở cửa chữa trị cho mọi thành phần, quân sự cũng như dân sự kể cả dân bản xứ. Năm 1925 Bệnh viện Quân sự chính thức sang tên "Bệnh viện Grall" để vinh danh Giám đốc Y tế Nam Kỳ, bác sĩ Charles Grall. Tháng Tư năm 1945 thời Đệ nhị Thế chiến bệnh viện bị trúng bom, phá sập mé phía bắc, tiêu hủy các phòng thí nghiệm.

In [ ]:
# df.to_json('data_gen/wikipedia_20220620_cleaned_parsed.jsonl', orient='records', lines=True)

# Title matching

In [26]:
import transformers
transformers.logging.set_verbosity_error()

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base',device='cuda')

No sentence-transformers model found with name C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
df = pd.read_json('data_gen/wikipedia_20220620_cleaned_parsed.jsonl', orient='records', lines=True)

In [5]:
titles = df['title'].parallel_apply(tokenize).tolist()
titles[:20]

['Nhơn_Đức',
 'Dobin am See',
 'Hyloniscus_rilensis',
 'Chesias_ornata',
 'Membrey',
 'Bộ Nông_nghiệp và Phát_triển nông_thôn ( Việt_Nam )',
 'Thành cổ Baku',
 'Digitaria_effusa',
 'Cyclodictyon_immersum',
 'Psychotria_nervosa',
 'Nodulotrophon_septus',
 'Griphapex_scutellaris',
 'Le_Mesnil - Amelot',
 'Bảng tổng_sắp huy_chương Thế vận_hội Mùa đông 2018',
 'Kauai',
 'Tirsa_fiona',
 'Teinobasis_filamenta',
 'Nippon_Steel',
 'Eublemma_minutata',
 'Valeri_Kaptilov']

In [6]:
pickle.dump(titles, open('data_gen/titles.pkl', 'wb'))

In [ ]:
# title_embeddings = model.encode(titles,convert_to_tensor=True,show_progress_bar=True,device='cuda')

In [ ]:
# pickle.dump(title_embeddings, open('data_gen/title_embeddings.pkl', 'wb'))

In [6]:
title_embeddings = pickle.load(open('data_gen/title_embeddings.pkl', 'rb'))

In [10]:
q_raw = '''huyện Tuy An'''
q = tokenize(q_raw)
query_embedding = model.encode(q,convert_to_tensor=True,)
print(q)
hits = util.semantic_search(query_embedding, title_embeddings, top_k=10)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']
    title = titles[corpus_id]
    print(f'{title} - {doc_score}')

huyện Tuy_An
Tuy_An - 0.9403116106987
An_Thọ , Tuy_An - 0.8956291675567627
An_Hiệp , Tuy_An - 0.8954504728317261
An_Cư , Tuy_An - 0.8625949621200562
An_Ninh_Đông , Tuy_An - 0.8604764938354492
An_Định , Tuy_An - 0.8470880389213562
An_Ninh_Tây , Tuy_An - 0.8405049443244934
An_Mỹ , Tuy_An - 0.8361471891403198
Phú_Yên ( xã ) - 0.7179363965988159
Phú_Yên - 0.7162452936172485


# Document Retriever

In [ ]:
def new_sentences(row):
    from pyvi.ViTokenizer import tokenize
    title=row['title']
    text=row['text']
    return [tokenize(title),text]

In [ ]:
sentences = df['text'].tolist()

In [12]:
len(sentences)

1273469

In [ ]:
# pickle.dump(sentences,open('data_gen/sentences.pkl','wb'))

In [11]:
sentences = pickle.load(open('data_gen/sentences.pkl','rb'))

In [ ]:
corpus_embeddings = model.encode(sentences,convert_to_tensor=True,show_progress_bar=True,device='cuda')

In [ ]:
# save corpus_embeddings
# pickle.dump(corpus_embeddings,open('data_gen/corpus_embeddings_2.pkl','wb'))

In [13]:
corpus_embeddings = pickle.load(open('data_gen/corpus_embeddings.pkl','rb'))

In [14]:
corpus_embeddings.shape

torch.Size([1273469, 768])

In [15]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

Some weights of the model checkpoint at VoVanPhuc/sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaForSequenceClassification: ['mlp.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at VoVanPhuc/sup-SimCSE-VietNamese-phobert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.b

In [16]:
with open('data/zac2022_train_merged_final.json',encoding='utf-8') as f:
    data=json.load(f)
    ddf = pd.json_normalize(data,'data')

In [17]:
ddf=ddf[ddf['title']!='']
ddf=ddf[ddf['category']=='FULL_ANNOTATION']

In [18]:
example = ddf.sample(1).iloc[0]

In [33]:
q_raw = example['question']
q = tokenize(q_raw)
query_embedding = model.encode(q,convert_to_tensor=True,)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)[0]

##### Re-Ranking #####
# Now, score all retrieved passages with the cross_encoder
cross_inp = [[q, sentences[hit['corpus_id']][1]] for hit in hits]
cross_scores = cross_encoder.predict(cross_inp)

# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]
# Output of top-5 hits from bi-encoder
print(q_raw)
print(example.title)
print(example.answer)
print("\n-------------------------\n")
print("Top-3 Bi-Encoder Retrieval hits")
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
for hit in hits[0:10]:
    print("\t{:.3f}\t{}".format(hit['score'], sentences[hit['corpus_id']][0]))

# Output of top-5 hits from re-ranker
print("\n-------------------------\n")
print("Top-3 Cross-Encoder Re-ranker hits")
hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
for hit in hits[0:10]:
    print("\t{:.3f}\t{}".format(hit['cross-score'], sentences[hit['corpus_id']][0]))

huyện Tịnh Biên của tỉnh an giang có rừng tràm nổi tiếng nào
An Giang
wiki/Rừng_tràm_Trà_Sư

-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.787	Tịnh_Biên
	0.767	Rừng tràm Trà_Sư
	0.766	Cửa_khẩu Tịnh_Biên
	0.748	Tịnh_Biên ( thị_trấn )
	0.748	Khu kinh_tế cửa_khẩu An_Giang
	0.730	Thới_Sơn ( định_hướng )
	0.719	An Nông , Tịnh_Biên
	0.714	An_Cư , Tịnh_Biên
	0.710	Núi_Dài Năm_Giếng
	0.708	An_Phú , Tịnh_Biên

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	0.500	Núi_Nước
	0.499	Chùa Thới_Sơn
	0.496	Cleistanthus_travancorensis
	0.496	Bảy_Núi
	0.494	Chà_chôi
	0.494	Lương_Phi
	0.494	Cleistanthus_namatanaiensis
	0.493	Núi_Sam
	0.492	Chaunus_gallardoi
	0.490	Khóm Tắc_Cậu
